## THE CIRCUIT CONFIGURATION

In [1]:
# check if notebook is in colab
try:
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass

import os
import json
import ezkl

import torch
from torchvision import transforms
from PIL import Image
from defineSNN import SiameseBambooNN


# Specify all the files we need
model_path = os.path.join('snn.onnx')
compiled_model_path = os.path.join('snn.ezkl')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')
srs_path = os.path.join('kzg.srs')
data_path = os.path.join('input1.json')

In [2]:
### Load the image pair as input ready for inference
class SiameseImageLoader:
    def __init__(self, transform=None):
        self.transform = transform

    def load_and_transform_pair(self, image1_path, image2_path):
        img1 = Image.open(image1_path).convert('L')
        img2 = Image.open(image2_path).convert('L')

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        img1 = img1.to(device)
        img2 = img2.to(device)

        return img1, img2

transform = transforms.Compose([
    transforms.Resize((10, 10)),
    transforms.ToTensor(),
])

# Example inputs (TrueNegative, TruePositive, FalseNegative)
# (FalseNegative were produced with a growing rate of bamboo higher than what the model was trained on)
siamese_loader = SiameseImageLoader(transform)
image1_path = "TestValid_t0.jpeg"
image2_path = "TestValid_t1.jpeg"
img1, img2 = siamese_loader.load_and_transform_pair(image1_path, image2_path) # Separate tensors for each image
img1 = img1.unsqueeze(0)
img2 = img2.unsqueeze(0)

print(img1.size())

torch.Size([1, 1, 10, 10])


In [3]:
# Load the trained nn model and make inference on test inputs
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loaded_model = SiameseBambooNN().to(device)
loaded_model.load_state_dict(torch.load('trainedSNN.pth', map_location=torch.device('cpu')))
loaded_model.eval()

with torch.no_grad():
    finalout = loaded_model(img1, img2)
    predicted_label = (torch.sigmoid(finalout) > 0.5).item()

print(f'Same class probability: {torch.sigmoid(finalout)}')
print(f'Predicted Label: {predicted_label}')

# Calculate the total number of parameters
total_params = sum(p.numel() for p in loaded_model.parameters())
print(f'Total Parameters: {total_params}')

Same class probability: tensor([[0.4207]])
Predicted Label: False
Total Parameters: 3393


In [4]:
# Export the loaded Siamese Network model to ONNX
torch.onnx.export(loaded_model,                      # loaded Siamese Network model
                  (img1, img2),                      # model input (or a tuple for multiple inputs)
                  'snn.onnx',                        # where to save the ONNX model
                  export_params=True,                # store the trained parameter weights inside the model file
                  opset_version=14,                  # the ONNX version to export the model to
                  do_constant_folding=True,          # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})



# Move tensors from GPU to CPU
img1_cpu = img1.cpu()
img2_cpu = img2.cpu()
print(img1_cpu.size())

# Serialize data into a JSON file
input_data = [
    img1_cpu.detach().numpy().reshape([-1]).tolist(),
    img2_cpu.detach().numpy().reshape([-1]).tolist(),
]

data = dict(input_data=input_data)
json.dump(data, open(data_path, 'w'))

torch.Size([1, 1, 10, 10])


In [5]:
#For ezkl to compute a snark, it needs some settings to determine how to create the circuit.
#This cell instantiates some parameters that determine the circuit shape, size etc

py_run_args = ezkl.PyRunArgs()
print('done pyrunargs')
py_run_args.input_visibility = "public"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "fixed" # "fixed" for params means that the committed to params are used for all proofs
py_run_args.variables = [("batch_size", 1)]

res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)
assert res == True

done pyrunargs


In [6]:
res = await ezkl.calibrate_settings(data_path, model_path, settings_path, "resources")
assert res == True

TypeError: object bool can't be used in 'await' expression

In [ ]:
# Now compile the model into a circuit
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

In [7]:
# get public srs from kzg ceremony, saved to srs path.
res = ezkl.get_srs(srs_path, settings_path)
assert res == True

In [8]:
# setup the circuit and make sure the keys are generated afterwards.
res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        srs_path,
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)

spawning module 2
spawning module 2


In [9]:
# GENERATE verifier associated with the circuit
try:
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "solc-select"])
    !solc-select install 0.8.20
    !solc-select use 0.8.20
    !solc --version

# rely on local installation if the notebook is not in colab
except:
    pass

In [10]:
sol_code_path = os.path.join('Verifier.sol')
abi_path = os.path.join('Verifier.abi')

res = ezkl.create_evm_verifier(
        vk_path,
        srs_path,
        settings_path,
        sol_code_path,
        abi_path
    )

assert res == True
assert os.path.isfile(sol_code_path)

## THE PROVER

In [11]:
witness_path = os.path.join('witness.json')
data_path = os.path.join('input1.json')

In [12]:
# now generate the witness file
res = ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

In [13]:
# !!!Generate the proof!!!
proof_path = os.path.join('proof.json')

proof = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        srs_path,
        "single",
    )

print(proof)
assert os.path.isfile(proof_path)

spawning module 2


In [56]:
# Sanity check off-chain verification
res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        srs_path,
    )

assert res == True
print("verified")

verified


In [57]:
onchain_input_array = []

# using a loop
# avoiding printing last comma
formatted_output = "["
for i, value in enumerate(proof["instances"]):
    for j, field_element in enumerate(value):
        onchain_input_array.append(ezkl.vecu64_to_felt(field_element))
        formatted_output += str(onchain_input_array[-1])
        if j != len(value) - 1:
            formatted_output += ", "
    formatted_output += "]"

# This will be the values you use onchain
# copy them over to remix and see if they verify
# What happens when you change a value?
print("pubInputs: ", formatted_output)
print("proof: ", "0x" + proof["proof"])


# Create a dictionary for the JSON data
json_data = {
    "pubInputs": formatted_output,
    "proof": "0x" + proof["proof"]
}

# Save the JSON data to a file
with open('evmInputs2.json', 'w') as json_file:
    json.dump(json_data, json_file, indent=2)


pubInputs:  [...]
proof:  ...
